In [1]:
%matplotlib inline
%load_ext zipline
%load_ext autoreload
%autoreload 2

In [1]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers

import plotly
import cufflinks

# (*) To communicate with Plotly's server, sign in with credentials file
import chart_studio.plotly as py

# (*) Useful Python/Plotly tools
import plotly.tools as tls   

# (*) Graph objects to piece together plots
from plotly.graph_objs import *

from zipline import run_algorithm
from zipline.utils.run_algo import load_extensions

from investment_analysis.markowitz import strategy
from investment_analysis.markowitz import computation
#from investment_analysis.symbols import VANGUARD_UNIVERSE

import investment_analysis

import pandas_datareader.data as web

load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

cufflinks.go_offline() #will make cufflinks offline
cufflinks.set_config_file(offline=False, world_readable=True)

# Turn off progress printing 
solvers.options['show_progress'] = False
os.environ['ZIPLINE_ROOT'] = '/home/rory/.zipline'

In [2]:
yf.Ticker('VOD.L').history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-03-22,128.039993,128.500000,126.760002,127.000000,195221303,0,0
2022-03-23,126.660004,127.139999,124.120003,124.260002,188296163,0,0
2022-03-24,124.620003,125.940002,123.959999,124.839996,99985046,0,0
2022-03-25,124.699997,125.587997,123.379997,125.320000,90350098,0,0
2022-03-28,125.620003,127.339996,125.300003,125.580002,54142860,0,0
2022-03-29,126.540001,127.519997,125.407997,126.559998,186340877,0,0
2022-03-30,127.080002,129.419998,126.620003,129.419998,242288918,0,0
2022-03-31,126.199997,126.940002,124.839996,124.839996,103090096,0,0
2022-04-01,125.360001,126.980003,124.519997,126.400002,109074831,0,0


In [ ]:
# get ohlc's
vger = yf.Ticker('VGER.L').history('5y')
vuke = yf.Ticker('VUKE.L').history('5y')
vmid = yf.Ticker('VMID.L').history('5y')
vusa = yf.Ticker('VUSA.L').history('5y')
vjpn = yf.Ticker('VJPN.L').history('5y')
vwrl = yf.Ticker('VWRL.L').history('5y')

In [ ]:
vixm = yf.Ticker('VIXM').history('5y')
svixm = yf.Ticker('SVXY').history('5y')


In [ ]:
vusa.Close.pct_change()

In [ ]:
returns = pd.DataFrame(list(map(
        lambda close: close.pct_change().dropna().squeeze(), [
            vger['Close'],
            vuke['Close'],
            vmid['Close'],
            vusa['Close'],
            vjpn['Close'],
            vwrl['Close'],
        ]
    )), ).T.dropna()

In [ ]:
returns.std()

In [ ]:
weights, _, _ = computation.optimal_portfolio(returns.T)
weights

In [ ]:
from zipline.algorithm import TradingAlgorithm

def analyze(context: TradingAlgorithm, perf):
    print(context.portfolio)
    
    fig = plt.figure(figsize=(20,10))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    bench_mark: pd.Series = context.benchmark_returns.squeeze().dropna()
    bench_mark = context.portfolio.starting_cash*bench_mark.add(1).cumprod()
    perf.portfolio_value.plot(ax=ax1, legend=True)
    bench_mark.plot(ax=ax1, legend=True)
    ax1.set_ylabel('portfolio value in $')
    perf_trans = perf.loc[[t != [] for t in perf.transactions]]
    buys = perf_trans.loc[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.loc[[t[0]['amount'] < 0 for t in perf_trans.transactions]]
    #buys.plot(ax=ax1)
    #sells.plot(ax=ax1)
    ax2.set_ylabel('weights')
    for asset in context.assets:
        perf[asset.symbol].plot(ax=ax2)
    plt.legend(loc=0)
    plt.show()

#analyze(Tr, perf)

In [ ]:
sp500 = web.DataReader('SP500', 'fred', start, end).SP500

In [ ]:


start = pd.Timestamp('2018-01-01')
end = pd.Timestamp('2022-04-15')

sp500 = web.DataReader('SP500', 'fred', start, end).SP500
benchmark_returns = sp500.pct_change()


perf: pd.DataFrame = run_algorithm(
    start=start.tz_localize('UTC'),
    end=end.tz_localize('UTC'),
    initialize=strategy.initialize,
    handle_data=strategy.handle_data,
    benchmark_returns=benchmark_returns,
    analyze=analyze,
    capital_base=27000,
    bundle='quandl',
    data_frequency='daily'
)
perf.to_csv('./backtest.pickle')

In [ ]:
perf[['VMID.L_20_mvavg', 'VMID.L_100_mvavg']].plot()

In [ ]:
from zipline.data import bundles
bundle = bundles.load('quandl')
assets = bundle.asset_finder.retrieve_all(bundle.asset_finder.sids)

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from investment_analysis.zipline_ingester import run_pipeline_against_bundle


pipeline = Pipeline({'close': USEquityPricing.close.latest})
out = run_pipeline_against_bundle(pipeline, '2012', '2013', bundle='quandl')
out.unstack()